In [1]:
from catalog import Local

from pprint import pprint

# Demo

## `Catalog`

Instantiate `Local` class and pass local bucket path:

In [2]:
catalog = Local('../bucket')

When remote, the `bucket` attribute represents the authenticated bucket object; when local, the path the directory serving as the bucket:

In [3]:
catalog.bucket

'../bucket'

Last main attribute `audiences` maps state file configurations and the data available in the bucket. This attribute behaves as a generator that instantiates the `Audience` class for each audience state file found:

In [4]:
catalog.audiences

<generator object Local._fetch_audiences at 0x1093e68c0>

When iterating, generator will yield one instance at a time, reducing the memory usage:

In [5]:
for audience in catalog.audiences:
    print(audience.name)

demo_audience


## `Audience`

In [6]:
from audience import Audience
# Reset catalog instance to reset `audiences` generator function and iterate through attribute again.
catalog = Local('../bucket')

Each Audience class instance represents the state of the give audience:

In [7]:
audience: Audience = next(catalog.audiences)
audience

Attributes store specific information:

In [8]:
audience.description

'A demo audience shared between two adtechs.'

`state` property represents the most up-to-date states of all audience components:

> Compare this state with the last state printed at the end of this demo.

In [9]:
pprint(audience.state)

{'demo_audience': {'adtechA': {'audience_type': 'TYPE_X',
                               'id': None,
                               'last_response': {'date': None,
                                                 'message': None,
                                                 'status': None},
                               'name': 'demo_audience'},
                   'adtechB': {'expiration_time': 300,
                               'id': None,
                               'last_response': {'date': None,
                                                 'message': None,
                                                 'status': None},
                               'name': 'demo_audience'},
                   'description': 'A demo audience shared between two adtechs.',
                   'source': {'endpoint': 'test/',
                              'last_response': {'date': '20231107',
                                                'message': 'Success: Demo '
                     

Three components are: `ApiGateway`, `AdtechA`, `AdtechB`. Each component is both an `Audience` instance attribute and a dedicated class instance:

In [10]:
print(audience.source)
print(audience.adtech_a)
print(audience.adtech_b)

## `Adtech`

Each adtech instance attribute will contain specific attributes of that specific adtech:

In [11]:
print(f'Type: {audience.adtech_a.audience_type} | Value: {audience.adtech_a.audience_type.value}')
print(f'Type: {audience.adtech_b.expiration_time} | Value: {audience.adtech_b.expiration_time.value}')

Type: AudienceType.TYPE_X | Value: TYPE_X
Type: <adtechs.adtechB.AdtechB.ExpirationTime object at 0x11918dd10> | Value: 300


`Status` indicates if audience is updloaded or not:

In [12]:
print(audience.adtech_a.status)
print(audience.adtech_b.status)

Status.NOT_POSTED
Status.NOT_POSTED


Payloads can be accessed via the attribute, when Status is `NOT_POSTED`:

In [13]:
pprint(audience.adtech_a.payload)
print('\n')
pprint(audience.adtech_b.payload)

{'data': [[{'emails': ['e6785a6df76f8b324a6153b506febf690b118d1f1a977da4a64b6e3c2ba57cfe'],
            'phoneNumbers': ['3e2425cbe51afa26dea57686b9d2fba24136b4aa8fe8d47216ccd45c1f0a51b3'],
            'zipCodes': ['8dbdccef2106bebd59fdf2b0fd147a2f17607e1c1b3a574122c14b153171858a']},
           {'emails': ['d151868068e2336d305809bf706becee99c79f0c71e016fb249b1ae2c643fee3'],
            'phoneNumbers': ['9ff62babcd637c1149f95f89725144fa40ce1d8a3cb9ce0cf00c580be97fc769'],
            'zipCodes': ['ab75305e5f7dd91ee5564f4136af3d63b1d125888928ee70f63df71432d5795c']}]],
 'description': 'A demo audience shared between two adtechs.',
 'name': 'demo_audience',
 'type': 'TYPE_X'}


{'data': [[['e6785a6df76f8b324a6153b506febf690b118d1f1a977da4a64b6e3c2ba57cfe'],
           ['3e2425cbe51afa26dea57686b9d2fba24136b4aa8fe8d47216ccd45c1f0a51b3'],
           ['8dbdccef2106bebd59fdf2b0fd147a2f17607e1c1b3a574122c14b153171858a']],
          [['d151868068e2336d305809bf706becee99c79f0c71e016fb249b1ae2c643f

## In Action

In [14]:
catalog = Local('../bucket')

for audience in catalog.audiences:
    if audience.adtech_a.status.value == 0:
        audience.adtech_a.upload()

    if audience.adtech_b.status.value == 0:
        audience.adtech_b.upload()

    print('\n')
    pprint(audience.state)
    catalog.push_state(audience)



{'demo_audience': {'adtechA': {'audience_type': 'TYPE_X',
                               'id': '123456789',
                               'last_response': {'date': '20231107',
                                                 'message': 'Success: Demo '
                                                            'response message.',
                                                 'status': 200},
                               'name': 'demo_audience'},
                   'adtechB': {'expiration_time': 300,
                               'id': '0987654321',
                               'last_response': {'date': '20231107',
                                                 'message': 'Success: Demo '
                                                            'response message.',
                                                 'status': 200},
                               'name': 'demo_audience'},
                   'description': 'A demo audience shared between two adtechs.',
     

Check for updates status:

In [15]:
# Instantiate bucket again to access generator:
catalog = Local('../bucket')

for audience in catalog.audiences:
    print(audience.adtech_a.status)
    print(audience.adtech_b.status)

Status.POSTED
Status.POSTED
